<a href="https://colab.research.google.com/github/JonasFerReis/Genetic-Algorithm-TSP/blob/main/genetic-algorithm-TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relatório - Problema do Caixeiro Viajante utilizando Algoritmos Genéticos

**Aluno:** Jonas Fernandes dos Reis  
**Matrícula:** 202020470

## Visão Geral

Dado um conjunto de cidades e suas respectivas coordenadas em um plano
bidimensional, o objetivo é encontrar a rota mais curta possível que visite cada cidade exatamente uma vez e retorne à cidade de origem. Esse problema é conhecido como Problema do Caixeiro Viajante (TSP).  
Com a utilização de Algoritmos Genéticos, definimos alguns conceitos:

* population: É um conjunto com todos os indíviduos.
* individual: É combinação de coordenadas, que representam uma rota entre as cidades.
* fitness: É a aptdão de um indíviduo, que significa a distância total da rota que ele representa.
* gene: É uma coordenada.

## Inicialização

Começamos definindo os parâmetros do algoritmo genético.

* population_size: Tamanho da população
* tournament_size: Tamanho do torneio
* generations: Quantidade de gerações
* crossover_rate: Taxa de cruzamento
* mutation_rate: Taxa de mutação

Em seguida é definido o conjunto de cidades que será utilizado, cada cidade é representado por suas coordenadas X e Y em um plano bidimensional.

```
cities = [
    (12, 15),
    (8, 20),
    (3, 7),
    (16, 11),
    (5, 18),
    (9, 2),
    (14, 6),
    (1, 13),
    (17, 4),
    (19, 10),
]
```
## Funções

### fitness:

```
def fitness(individual):
    distance = 0
    for i in range(len(individual) - 1):
        deltaX = individual[i + 1][0] - individual[i][0]
        deltaY = individual[i + 1][1] - individual[i][1]
        distance += math.sqrt((deltaX ** 2) + (deltaY ** 2))

    deltaX = individual[len(individual) - 1][0] - individual[0][0]
    deltaY = individual[len(individual) - 1][1] - individual[0][1]
    distance += math.sqrt((deltaX ** 2) + (deltaY ** 2))

    return round(distance, 2)
```
Função onde é calculado a aptdão dos individuos. Iteramos pelas coordenadas de um individuo, calculando a Distância Euclidiana entre cidades consecutivas e somando os resultados, ao final calculamos a distância da última cidade até a primeira. Por fim retornamos a distância total da rota.

### crossover:

```
def crossover(parent1, parent2):
    cut_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:cut_point]
    child2 = parent2[:cut_point]

    for i in range(len(parent2)):
        if parent2[i] not in child1:
            child1.append(parent2[i])

    for i in range(len(parent1)):
        if parent1[i] not in child2:
            child2.append(parent1[i])

    return child1, child2
```
Função onde realizamos os cruzamentos entre pais, utilizando o Single Point Crossover. Definimos um ponto de corte nos pais, e geramos dois decendentes:

* Descendente 1: Recebe a primeira metade do primeiro pai.
* Descendente 2: Recebe a primeira metade do segundo pai.

Em seguida completamos a segunda metade de cada decendentes com partes do pai oposto, de maneira que não tenha coordenadas repetidas.  
Por fim retornamos os descendentes.

### mutation:

```
def mutation(individual):
    for i in range(len(individual)):
        if random.random() <= mutation_rate:
            x = individual.index(random.choice(individual))
            while (i == x):
                x = individual.index(random.choice(individual))
            individual[i], individual[x] = individual[x], individual[i]
    return individual
```
Função onde realizamos mutação nos individuos. Iteramos pelos genes de um individuo, verificando se a mutação vai ocorrer naquele gene ou não, de acordo com o parâmetro `mutation_rate`. Caso ela aconteça, então selecionamos aleatóriamente outro gene do individuo, verificamos se o gene escolhido não é o mesmo, e o trocamos de lugar com o gene atual.

### genetic_algorithm:
```
def genetic_algorithm():
    population = []
    best_fitness = float("-inf")
    best_individual = None

    # Inicialização da população
    for _ in range(population_size):
        individual = random.sample(cities, len(cities))
        population.append(individual)

    # Evolução das gerações
    for _ in range(generations):

        # Avaliação de aptidão
        population_fitness = []
        for i in range(population_size):
            population_fitness.append(fitness(population[i]))
        min_fitness = min(population_fitness)
        if min_fitness > best_fitness:
            best_fitness = min_fitness
            best_individual = population[population_fitness.index(min_fitness)].copy()

        # Seleção por torneio
        parents = []
        for _ in range(population_size):
            competitors = random.sample(population_fitness, tournament_size)
            winner = min(competitors)
            parents.append(population[population_fitness.index(winner)])

        # Cruzamento
        for _ in range(population_size):
            parent1 = random.choice(parents)
            parent2 = random.choice(parents)
            while (parent1 == parent2):
                parent2 = random.choice(parents)
            if random.random() <= crossover_rate:
                child1, child2 = crossover(parent1, parent2)
                if fitness(child1) < max(population_fitness):
                    population[population_fitness.index(min(population_fitness))] = child1
                if fitness(child2) < max(population_fitness):
                    population[population_fitness.index(min(population_fitness))] = child2

        # Mutação
        for i in range(population_size):
            population[i] = mutation(population[i])

    return best_individual, best_fitness
```
O Algoritmo genético, onde será realizado toda a evolução. Começamos definindo as variavéis `population`, `best_fitness` e `best_individual`.

#### Inicialização da população:

Preenchemos o vetor `population` com combinações aleatórias das cidades.

#### Evolução das gerações:

Um loop, controlado pelo parâmetro `generations`, onde a cada iteração a população atual é evoluida. A evolução das gerações consistem nos seguintes passos:

* Avaliação de aptdão:

Iteramos pelo vetor `population`, calculando a aptdão de cada individuo e armazenando no vetor `population_fitness`. Em seguida verificamos qual o menor fitness desse vetor e salvamos seu valor em `best_fitness` e o individuo que possui esse valor em `best_individual`.

* Seleção por Torneio:

Aqui realizamos a seleção dos pais para criar descendentes para a próxima geração, selecionamos um conjunto de pais com o tamanho da própria população. Começamos definindo aleatóriamente um conjunto de competidores do vetor `population_fitness`, depois verificamos qual o menor fitness dos competidores e o declaramos como vencedor, em seguida adicionamos no vetor `parents` o individuo dono desse fitness.

* Cruzamento:

Depois de selecionar os pais, realizamos o cruzamento entre eles. Primeiro selecionamos aleatóriamente 2 pais do vetor `parents` e garantimos que não sejam iguais. Depois verificamos se o cruzamento vai acontecer ou não, de acordo com o parâmetro `crossover_rate`. Caso positivo, então utilizamos a função `crossover()` para gerar dois descendentes. Por último calculamos a aptdão dos descendentes e substituimos as rotas menos aptas da população atual pelos novos descendentes.

* Mutação:

Última etapa da evolução da geração. Depois de atualizar a população atual com novos descendentes, iteramos pela população, executando a função `mutation()` para cada individuo.

## Resultados

Os testes foram realizados considerando um conjunto com 10 cidades, o que nos dá 3628800 possibilidades de rotas diferentes. Para cada alteração nos parâmetros, o algoritmo era executado 100 vezes. Com isso obtivemos os seguintes resultados:

Cidades utilizadas:
```
cities = [
    (12, 15),
    (8, 20),
    (3, 7),
    (16, 11),
    (5, 18),
    (9, 2),
    (14, 6),
    (1, 13),
    (17, 4),
    (19, 10),
]
```
Parâmetros iniciais:
```
population_size = 100
tournament_size = 10
generations = 100
```
Com esses parâmetros, os resultados foram que o caminho com o menor custo era de 88.67 unidades. Porém durante as 100 execuções do algoritmo o valor que aparecia com maior frequência foi 92.66 unidades.

Alterando o parâmetro `generations` para 1000, obtivemos resultado piores que o anterior, onde após 100 execuções do algoritmo o melhor valor encontrado foi de 93.09 unidades e o valor que aparecia com maior frequência era de 95.19 unidades e um tempo de execução de em média 4 segundos.

Voltando o parâmetro `generations` novamente para 100, e aumentando `population_size` para 1000, obtivemos uma considerável melhora nos resultados, onde após 100 execuções, o melhor valor encontrado foi de 79.15 e o valor com maior frequência foi de 81.55 unidades, e um tempo de execução de em média 10 segundos por execução.

Mantendo o `populations_size` em 1000 e aumentando o tournament_size para 100, obtivemos resultados muito parecidos com os anteriores, onde o melhor valor encontrado foi de 79.77 e o valor com maior frequência foi de 81.11 unidades. Porém tivemos um consideravel aumento no tempo de execução do algoritmo que foi de em média 20 segundos por execução.

Alterações nos parâmetros `crossover_rate` e `mutations_rate`, provocavam uma piora no resultado, ou uma mudança pouco significativa em comparação ao outros parâmetros, então os valores padrões de 0.6 e 0.1 foram a melhor escolha em na maioria dos casos.

Com isso podemos observar que o parâmetro de maior influência, para esse problema, é o `population_size`. Isso se deve ao fato, de que temos uma grande quantidade de possibilidades de rotas, que para 10 cidades são mais de três milhões de rotas possiveis. Com uma baixa população, por exemplo igual a 100, pegamos um conjunto muito pequeno do total disponivel e realizamos a evolução considerando apenas essa parcela das possibilidades. Ao aumentar o tamanho da população conseguimos abranger um conjunto muito maior de possibilidades e evoluir ele, porém com o custo de ter um aumento significativo no tempo de execução. Formas de otimização podem ser estudadas e implementadas para melhorar esse caso.




In [ ]:

import random
import math

# Parâmetros do algoritmo genético
population_size = 100
tournament_size = 10
generations = 100
crossover_rate = 0.6
mutation_rate = 0.1

# Representação das cidades por coordenadas X e Y
cities = [
    (12, 15),
    (8, 20),
    (3, 7),
    (16, 11),
    (5, 18),
    (9, 2),
    (14, 6),
    (1, 13),
    (17, 4),
    (19, 10),
]

# Função de aptidão (fitness function)
def fitness(individual):
    distance = 0
    for i in range(len(individual) - 1):
        # Distancia Euclidiana
        deltaX = individual[i + 1][0] - individual[i][0]
        deltaY = individual[i + 1][1] - individual[i][1]
        distance += math.sqrt((deltaX ** 2) + (deltaY ** 2))

    # Distancia Euclidiana da última cidade até a primeira
    deltaX = individual[len(individual) - 1][0] - individual[0][0]
    deltaY = individual[len(individual) - 1][1] - individual[0][1]
    distance += math.sqrt((deltaX ** 2) + (deltaY ** 2))

    return round(distance, 2)

# Função de cruzamento (crossover)
def crossover(parent1, parent2):
    # Ponto de corte para o cruzamento
    cut_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:cut_point]
    child2 = parent2[:cut_point]

    for i in range(len(parent2)):
        if parent2[i] not in child1:
            child1.append(parent2[i])

    for i in range(len(parent1)):
        if parent1[i] not in child2:
            child2.append(parent1[i])

    return child1, child2

# Função de mutação
def mutation(individual):
    for i in range(len(individual)):
        if random.random() <= mutation_rate:
            x = individual.index(random.choice(individual))
            while (i == x):
                x = individual.index(random.choice(individual))
            individual[i], individual[x] = individual[x], individual[i]
    return individual

# Algoritmo genético
def genetic_algorithm():
    population = []
    best_fitness = float("-inf")
    best_individual = None

    # Inicialização da população
    for _ in range(population_size):
        individual = random.sample(cities, len(cities))
        population.append(individual)

    # Evolução das gerações
    for _ in range(generations):

        # Avaliação de aptidão
        population_fitness = []
        for i in range(population_size):
            population_fitness.append(fitness(population[i]))

        min_fitness = min(population_fitness)
        if min_fitness > best_fitness:
            best_fitness = min_fitness
            best_individual = population[population_fitness.index(min_fitness)].copy()

        # Seleção por torneio
        parents = []
        for _ in range(population_size):
            competitors = random.sample(population_fitness, tournament_size)
            winner = min(competitors)
            parents.append(population[population_fitness.index(winner)])

        # Cruzamento
        for _ in range(population_size):
            parent1 = random.choice(parents)
            parent2 = random.choice(parents)
            while (parent1 == parent2):
                parent2 = random.choice(parents)
            if random.random() <= crossover_rate:
                child1, child2 = crossover(parent1, parent2)
                if fitness(child1) < max(population_fitness):
                    population[population_fitness.index(min(population_fitness))] = child1
                if fitness(child2) < max(population_fitness):
                    population[population_fitness.index(min(population_fitness))] = child2

        # Mutação
        for i in range(population_size):
            population[i] = mutation(population[i])

    return best_individual, best_fitness

# Resultados
best_solution, best_fitness = genetic_algorithm()
print("Melhor solução encontrada:", best_solution)
print("Melhor aptidão encontrada:", best_fitness)